# Aula 1

## Vídeo 1.2 - Biblioteca e os dados

GeoPandas é uma biblioteca de código aberto em Python que facilita a manipulação e análise de dados geoespaciais. Ela estende a funcionalidade da biblioteca pandas, amplamente usada para manipulação de dados, adicionando suporte para operações geoespaciais. Isso inclui a capacidade de ler e escrever diferentes formatos de arquivo geoespaciais (como Shapefiles, GeoJSON, etc.), realizar operações geoespaciais (como união espacial, interseção, etc.), e criar visualizações geoespaciais. A GeoPandas integra-se bem com outras bibliotecas Python amplamente utilizadas, como Matplotlib para visualização e NumPy para manipulação de dados numéricos. Ao proporcionar uma interface de alto nível e fácil de usar para operações geoespaciais complexas, a GeoPandas tornou-se uma ferramenta importante para geocientistas, urbanistas e outros que trabalham com dados geoespaciais.

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd

https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/15774-malhas.html

In [ ]:
url = 'https://github.com/alura-cursos/curso_geopandas/raw/main/dados/RJ_Municipios_2022.shp'

In [ ]:
rj = gpd.read_file(url)

In [ ]:
type(rj)

In [ ]:
rj.head()

In [ ]:
rj.plot()

In [ ]:
rj.plot(color='white',edgecolor='black',figsize=(15,8))

## Vídeo 1.3 - Trabalhando com Shapefiles

Shapefiles são um formato de arquivo de dados geoespaciais desenvolvido pela Esri, uma das principais empresas de software de Sistemas de Informação Geográfica (SIG). Eles são amplamente utilizados para armazenar dados de localização geográfica e atributos associados a essa localização. Um shapefile na verdade consiste em um conjunto de arquivos correlacionados (.shp, .shx, .dbf, e potencialmente outros) que juntos expressam vetores geoespaciais - pontos, linhas ou polígonos - juntamente com informações atributivas, que podem incluir detalhes como nomes, categorias e outras características associadas a cada entidade geográfica. Shapefiles são comumente usados em uma variedade de campos, incluindo urbanismo, geografia, logística, e muitos outros que requerem a análise e a visualização de dados geoespaciais.

In [ ]:
rj.plot(color='white',edgecolor='black',figsize=(15,8))

In [ ]:
rj = rj[rj['NM_MUN']=='Rio de Janeiro']

In [ ]:
rj

In [ ]:
rj.plot(color='orange',edgecolor='black',figsize=(15,8))

In [ ]:
rj.to_file('RJ-MUNIC.shp')

In [ ]:
rj_munic = gpd.read_file('RJ-MUNIC.shp')

In [ ]:
rj_munic.plot(color='orange',edgecolor='black',figsize=(15,8))

# Aula 2 - Sistemas de coordenadas


## Vídeo 2.1 - Preparando o dataframe

Latitude e longitude são utilizadas para localizar qualquer ponto na Terra. Latitude é uma medida de graus ao norte ou ao sul do equador, que é definido como 0 grau, com os pólos norte e sul sendo +90 e -90 graus, respectivamente. Longitude, por outro lado, mede graus a leste ou oeste do meridiano principal, conhecido como Meridiano de Greenwich, que é definido como 0 grau. Os graus de longitude vão de -180 a +180. Combinando uma coordenada de latitude e uma de longitude, podemos especificar a localização exata de qualquer ponto na Terra.

![](https://educa.ibge.gov.br/images/vamoscontar/atividades/mapa_mundo_coordenadas.jpg)

Fonte: https://educa.ibge.gov.br/professores/educa-atividades/17650-linhas-imaginarias.html

In [ ]:
import pandas as pd

In [ ]:
url = 'https://raw.githubusercontent.com/alura-cursos/curso_geopandas/main/dados/dados.csv'
df = pd.read_csv(url,sep='\t')

In [ ]:
df.head()

In [ ]:
type(df)

In [ ]:
x = zip(df['Longitude'],df['Latitude'])
x

In [ ]:
list(x)

In [ ]:
from shapely import Point

In [ ]:
pontos_geometricos = [Point(x) for x in zip(df['Longitude'],df['Latitude'])]

In [ ]:
pontos_geometricos

In [ ]:
crs = {'proj': 'latlong', 'ellps':'WGS84', 'datum': 'WGS84','no_defs':True}
geo_dados = gpd.GeoDataFrame(df, crs=crs, geometry=pontos_geometricos)

In [ ]:
geo_dados.head()

In [ ]:
geo_dados.to_file('DATASET.shp')

In [ ]:
geo_dados.plot(alpha=0.2)

## Vídeo 2.2 - Sistema de coordenadas

In [ ]:
rj.crs

In [ ]:
geo_dados.crs

## Vídeo 2.3 - Modificando o sistemas de coordenadas

🤖 Qual a zona UTM em que o município do Rio de Janeiro está inserido?

R: Zona UTM 23S

🤖 Qual o EPSG para zona UTM 23S utilizando o SIRGAS 2000?

R: EPSG 31983

https://www.spatialreference.org/

In [ ]:
rj = rj.to_crs('+proj=utm +zone=23 +south +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=km +no_defs')

In [ ]:
geo_dados = geo_dados.to_crs('+proj=utm +zone=23 +south +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=km +no_defs')

In [ ]:
rj.crs

In [ ]:
geo_dados.crs

In [ ]:
rj.plot()

# Aula 3 - Filtrando imóveis por região

## Vídeo 3.1 - Identificando outliers

In [ ]:
base = rj.plot(color='black',edgecolor='black',figsize=(15,10))
geo_dados.plot(ax=base,figsize=(15,10),alpha=0.2)

In [ ]:
rj.iloc[0].geometry

In [ ]:
geo_dados.iloc[0].geometry

In [ ]:
geo_dados.iloc[0].geometry.within(rj.iloc[0].geometry)

## Vídeo 3.2 - Selecionando dados em uma região

In [ ]:
geo_dados['geometry'].within(rj.iloc[0].geometry)

In [ ]:
geo_dados.shape[0]

In [ ]:
geo_dados = geo_dados[geo_dados['geometry'].within(rj.iloc[0].geometry)]

In [ ]:
geo_dados.shape[0]

In [ ]:
base = rj.plot(color='white',edgecolor='black',figsize=(15,10))
geo_dados.plot(ax=base,figsize=(15,10),alpha=0.2)

## Vídeo 3.3 - Acrescentando estações do metrô

https://www.data.rio/

https://www.data.rio/datasets/PCRJ::esta%C3%A7%C3%B5es-metr%C3%B4/explore?location=-22.843796%2C-43.262511%2C11.95

In [ ]:
url_m = 'https://github.com/alura-cursos/curso_geopandas/raw/main/dados/metro.geojson'

In [ ]:
metro = gpd.read_file(url_m)

In [ ]:
metro.crs

In [ ]:
metro = metro.to_crs('+proj=utm +zone=23 +south +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=km +no_defs')

In [ ]:
metro.crs

In [ ]:
base = rj.plot(color='white',edgecolor='black',figsize=(15,10))
geo_dados.plot(ax=base,figsize=(15,10),alpha=0.2)
metro.plot(ax=base, color='red',markersize=50)

# Aula 4 - Calculando distâncias

## Vídeo 4.1 - Estação do metrô mais próxima

In [ ]:
geo_dados.iloc[0].geometry

In [ ]:
distancias = metro.distance(geo_dados.iloc[0].geometry)

In [ ]:
type(distancias)

In [ ]:
distancias.min()

In [ ]:
geo_dados['Dist_metro'] = geo_dados['geometry'].apply(lambda x: metro.distance(x).min())

In [ ]:
geo_dados.head()

In [ ]:
import numpy as np
geo_dados_num = geo_dados.select_dtypes(include=[np.number])
correlacao = geo_dados_num.corr()

In [ ]:
import seaborn as sns
sns.heatmap(correlacao, annot=True, cmap='RdYlBu', linewidths=0.5)

## Vídeo 4.2 - Agregando dados de transportes

In [ ]:
url_t = 'https://github.com/alura-cursos/curso_geopandas/raw/main/dados/trem.geojson'

In [ ]:
trem = gpd.read_file(url_t)
trem = trem.to_crs('+proj=utm +zone=23 +south +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=km +no_defs')

In [ ]:
url_b = 'https://github.com/alura-cursos/curso_geopandas/raw/main/dados/brt.geojson'

brt = gpd.read_file(url_b)
brt = brt.to_crs('+proj=utm +zone=23 +south +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=km +no_defs')

In [ ]:
trem = trem[trem.within(rj.iloc[0].geometry)]
brt = brt[brt.within(rj.iloc[0].geometry)]

In [ ]:
base = rj.plot(color='white',edgecolor='black',figsize=(15,10))
geo_dados.plot(ax=base,figsize=(15,10),alpha=0.2)
metro.plot(ax=base, color='#b60a1c',markersize=50)
trem.plot(ax=base, color='#e39802',markersize=50)
brt.plot(ax=base, color='#309143',markersize=50)

base.legend(['Imóveis','Metrô','Trem','BRT'],loc='upper left')

In [ ]:
transporte = pd.concat([trem.geometry,metro.geometry,brt.geometry],ignore_index = True)

In [ ]:
type(transporte)

In [ ]:
geo_dados['Dist_Transporte'] = geo_dados['geometry'].apply(lambda x: transporte.distance(x).min())

In [ ]:
geo_dados.head()

In [ ]:
geo_dados_num = geo_dados.select_dtypes(include=[np.number])
correlacao = geo_dados_num.corr()

In [ ]:
sns.heatmap(correlacao, annot=True, cmap='RdYlBu', linewidths=0.5)

## Vídeo 4.3 - Calculando a distância até a praia

In [ ]:
url_p = '/content/drive/MyDrive/Alura/Cursos/3060 - Dados Geográficos: Geopandas/Cobertura_Vegetal_e_Uso_da_Terra_2018.geojson'

In [ ]:
praias = gpd.read_file(url_p)

In [ ]:
praias.head()

In [ ]:
praias.crs

In [ ]:
praias = praias.to_crs('+proj=utm +zone=23 +south +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=km +no_defs')

In [ ]:
praias.crs

In [ ]:
praias.plot(color='white',edgecolor='black',figsize=(15,8))

In [ ]:
praias['classe'].unique()

In [ ]:
praias = praias[praias['classe'] == 'Praia']

In [ ]:
praias.plot(color='white',edgecolor='black',figsize=(15,8))

In [ ]:
base = rj.plot(color='white',edgecolor='black',alpha=0.4,figsize=(15,10))
praias.plot(ax=base,color='white',edgecolor='black')
geo_dados.plot(ax=base,figsize=(15,10),alpha=0.2)

In [ ]:
geo_dados['Dist_Praia'] = geo_dados['geometry'].apply(lambda x: praias.distance(x).min())

In [ ]:
geo_dados_num = geo_dados.select_dtypes(include=[np.number])
correlacao = geo_dados_num.corr()

In [ ]:
sns.heatmap(correlacao, annot=True, cmap='RdYlBu', linewidths=0.5)

In [ ]:
sns.scatterplot(x='Dist_Praia',y='Valor',data=geo_dados)